<a href="https://colab.research.google.com/github/koklengyeo/aviation/blob/main/aviation_contracted_air_freight_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREAMBLE

In [23]:
#FOR FUTURE REFERENCE
#DOWNGRADE PYTHON TO VERSION USED DURING DEVELOPMENT
#!apt-get update -y
#!apt-get install python3.10 python3.10-distutils
#!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
#!update-alternatives --config python3
#!apt-get install python3-pip
#!python3 -m pip install --upgrade pip --user

In [24]:
#FOR FUTURE REFERENCE
#DOWNGRADE PACKAGES TO VERSION USED DURING DEVELOPMENT
#!pip install IPython==7.34.0
#!pip install ipywidgets==7.7.1
#!pip install matplotlib==3.7.1
#!pip install numpy==1.25.2
#!pip install pandas==2.0.3

In [25]:
#IMPORT PACKAGES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import *
from ipywidgets import *

# USER INPUT FORM (WIDGETS)

In [26]:
#DEFINE WIDGETS USED TO CAPTURE USER INPUTS

#BASE YEAR
#MUST BE BETWEEN 2019 AND PRESENT YEAR
by_widget=widgets.BoundedIntText(
    value=datetime.now().year,
    min=2019,
    max=datetime.now().year,
    step=1,
    description="Base Year",
    disabled=False)

#TARGET YEAR
#MUST BE BETWEEN (PRESENT YEAR+5) AND 2050
ty_widget=widgets.BoundedIntText(
    value=datetime.now().year+5,
    min=datetime.now().year+5,
    max=2050,
    step=1,
    description="Target Year",
    disabled=False)

#TYPE OF EMISSIONS DATA
ed_type_widget=widgets.RadioButtons(
    value=None,
    options=[
        ("Well-to-Wake","wtw"),
         ("Tank-to-Wake","ttw")],
    description="Type of Emissions Data",
    disabled=False)

#CONTRACTED AIR FREIGHT EMISSIONS IN TCO2E
#MUST BE >=0
cfe_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Contracted Air Freight Emissions in tCO2e",
    disabled=False)

#BASE YEAR TOTAL FLOWN CONTRACTED FREIGHT IN RTK
#MUST BE >=0
by_cfa_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Total Flown Contracted Freight in RTK",
    disabled=False)

#TYPE OF ACTIVITY FORECAST
af_type_widget=widgets.RadioButtons(
    value=None,
    options=[
        ("Contracted Air Freight CAGR","cagr"),
         ("Target Year Contracted Air Freight Activity","tya")],
    description="Type of Activity Forecast",
    disabled=False)

#CONTRACTED AIR FREIGHT CAGR
cagr_widget=widgets.FloatText(
    value=0,
    step=0.1,
    description="Contracted Air Freight CAGR in %",
    disabled=False)

#TARGET YEAR TOTAL FLOWN CONTRACTED FREIGHT IN RTK
#MUST BE >=0
ty_cfa_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Total Flown Contracted Freight in RTK",
    disabled=False)

#INVISIBLE BOOLEAN WIDGET TO REGISTER FIRST CLICK OF "CALCULATE TARGET" BUTTON
fc_widget=widgets.Valid(
    value=False,
    layout={"display":"none"})

In [27]:
#ENABLE/DISABLE CAGR & TY_CFA WIDGETS DEPENDING ON AF_TYPE

def update_widgets(*args):
  if af_type_widget.value=="cagr":
    #ENABLE CAGR
    cagr_widget.disabled=False
    #DISABLE TY_PA
    ty_cfa_widget.value=0
    ty_cfa_widget.disabled=True
  elif af_type_widget.value=="tya":
    #DISABLE CAGR
    cagr_widget.value=0
    cagr_widget.disabled=True
    #ENABLE TY_PA
    ty_cfa_widget.disabled=False
af_type_widget.observe(update_widgets,"value")

In [28]:
#DEFINE WIDGETS USED TO DISPLAY TEXT

overall_header=widgets.HTML(
    value="<h1>TARGET SETTING FOR CONTRACTED AIR FREIGHT</h1>\
    <h3>Version 2.0 (Feb-23)</h3>This is intended for companies using \
    contracted air freight services to set targets on scope 3 category 4 \
    'Upstream Transportation and Distribution' and scope 3 category 9 \
    'Downstream Transportation and Distribution' emissions. Targets are set on \
    a well-to-wheel basis and aligned with a 1.5&deg;C scenario.")
basic_info_header=widgets.HTML(
    value="<h2>1. Basic Information</h2>Please indicate the base year and \
    target year. The base year should be the most recent year with a complete \
    GHG inventory. Due to COVID considerations, the base year cannot be in the \
    period of 2020 to 2022. Target must cover a minimum of 5 years and a \
    maximum of 10 years from the year it is submitted to the SBTi for \
    validation.")
by_emissions_header=widgets.HTML(
    value="<h2>2. Base Year Emissions</h2>Please provide the well-to-wake \
    emissions in the base year. If this is not available, please provide the \
    tank-to-wake emissions instead. In this case, well-to-wake emissions will \
    be projected from tank-to-wake emissions.")
by_activity_header=widgets.HTML(
    value="<h2>3. Base Year Activity</h2>Please provide the contracted air \
    freight activity in the base year.")
ty_activity_header=widgets.HTML(
    value="<h2>4. Target Year Activity</h2>Please provide either the expected \
    CAGR for contracted air freight or the expected contracted air freight \
    activity in the target year.")
ty_activity_footer=widgets.HTML(
    value="Please ensure that all provided information is as complete and \
    accurate as possible. Click the button below to calculate the target.")
results_header=widgets.HTML(
    value="<h2>5. Target Modelling Results</h2>")

#ERROR MESSAGES IF USER INPUTS DO NOT MEET CERTAIN CONDITIONS
no_error=widgets.HTML()
by_error=widgets.HTML(
    value="<p style='color:red;'>Base Year: This cannot be in the period of \
      2020 to 2022.</p>")
ed_type_error=widgets.HTML(
    value="<p style='color:red;'>Type of Emissions Data: Please select the \
      applicable option.</p>")
cfe_error=widgets.HTML(
    value="<p style='color:red;'>Contracted Air Freight Emissions in tCO2e \
      must be provided.</p>")
cfa_error=widgets.HTML(
    value="<p style='color:red;'>Total Flown Contracted Freight in RTK must be \
      provided.</p>")
af_type_error=widgets.HTML(
    value="<p style='color:red;'>Type of Activity Forecast: Please select the \
      applicable option.</p>")
user_inputs_error=widgets.HTML(
    value="<p style='color:red;'>Information provided incomplete and/or \
      invalid. Please check and try again.</p>")
no_target_error=widgets.HTML(
    value="<p style='color:red;'>No target calculated. The base year intensity \
      is already below the 2050 level in the 1.5&deg;C scenario.</p>")

In [29]:
#WIDGET LAYOUT & STYLE

#SELECTION WIDGETS
layout={"width":"1100px"}
style={"description_width":"600px"}
ed_type_widget.layout=layout
ed_type_widget.style=style
af_type_widget.layout=layout
af_type_widget.style=style

#YEAR WIDGETS
layout={"width":"700px"}
style={"description_width":"600px"}
by_widget.layout=layout
by_widget.style=style
ty_widget.layout=layout
ty_widget.style=style

#EMISSIONS WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
cfe_widget.layout=layout
cfe_widget.style=style

#ACTIVITY WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
by_cfa_widget.layout=layout
by_cfa_widget.style=style
ty_cfa_widget.layout=layout
ty_cfa_widget.style=style

#PERCENTAGE WIDGETS
layout={"width":"700px"}
style={"description_width":"600px"}
cagr_widget.layout=layout
cagr_widget.style=style

#TEXT WIDGETS
layout={"width":"1100px"}
overall_header.layout=layout
basic_info_header.layout=layout
by_emissions_header.layout=layout
by_activity_header.layout=layout
ty_activity_header.layout=layout
ty_activity_footer.layout=layout
results_header.layout=layout
by_error.layout=layout
ed_type_error.layout=layout
cfe_error.layout=layout
cfa_error.layout=layout
af_type_error.layout=layout
user_inputs_error.layout=layout
no_target_error.layout=layout

# USER INPUT FORM (INTERACTIVITY)

In [30]:
#USER INPUT FORM - BASIC INFORMATION

def basic_info_check(by,
                     ty,
                     fc):
  basic_info_valid=True
  update_display(no_error,display_id="basic_info_check")
  #BY - CANNOT BE IN THE PERIOD OF 2020 TO 2022
  if by>=2020 and by<=2022:
    basic_info_valid=False
    if fc:
      display(by_error,display_id="basic_info_check")
  return basic_info_valid
basic_info_input=interactive(
    basic_info_check,
    by=by_widget,
    ty=ty_widget,
    fc=fc_widget)

In [31]:
#USER INPUT FORM - BASE YEAR EMISSIONS

def by_emissions_check(ed_type,
                       cfe,
                       fc):
  by_emissions_valid=True
  update_display(no_error,display_id="by_emissions_check")
  #ED_TYPE - USER MUST SELECT APPLICABLE OPTION
  if ed_type is None:
    by_emissions_valid=False
    if fc:
      display(ed_type_error,display_id="by_emissions_check")
  #CFE - MUST BE PROVIDED
  if cfe==0:
    by_emissions_valid=False
    if fc:
      display(cfe_error,display_id="by_emissions_check")
  return by_emissions_valid
by_emissions_input=interactive(
    by_emissions_check,
    ed_type=ed_type_widget,
    cfe=cfe_widget,
    fc=fc_widget)

In [32]:
#USER INPUT FORM - BASE YEAR ACTIVITY

def by_activity_check(by_cfa,
                      fc):
  by_activity_valid=True
  update_display(no_error,display_id="by_activity_check")
  #BY_CFA - MUST BE PROVIDED
  if by_cfa==0:
    by_activity_valid=False
    if fc:
      display(cfa_error,display_id="by_activity_check")
  return by_activity_valid
by_activity_input=interactive(
    by_activity_check,
    by_cfa=by_cfa_widget,
    fc=fc_widget)

In [33]:
#USER INPUT FORM - TARGET YEAR ACTIVITY

def ty_activity_check(af_type,
                      cagr,
                      ty_cfa,
                      fc):
  ty_activity_valid=True
  update_display(no_error,display_id="ty_activity_check")
  #AF_TYPE - USER MUST SELECT APPLICABLE OPTION
  if af_type is None:
    ty_activity_valid=False
    if fc:
      display(af_type_error,display_id="ty_activity_check")
  if af_type=="tya":
    #TY_CFA - MUST BE PROVIDED
    if ty_cfa==0:
      ty_activity_valid=False
      if fc:
        display(cfa_error,display_id="ty_activity_check")
  return ty_activity_valid
ty_activity_input=interactive(
    ty_activity_check,
    af_type=af_type_widget,
    cagr=cagr_widget,
    ty_cfa=ty_cfa_widget,
    fc=fc_widget)

# TARGET MODEL (DATA & PARAMETERS)

In [34]:
#IMPORT DATA

path="https://raw.githubusercontent.com/koklengyeo/migration/main/"

#IMPORT SECTOR DATA (PACE)
pace=pd.read_csv(
    path+"aviation_pace.csv",
    decimal=",",
    index_col=0,
    sep=";")
pace.columns=pace.columns.map(int)

#IMPORT SECTOR DATA (SDS)
sds=pd.read_csv(
    path+"aviation_sds.csv",
    decimal=",",
    index_col=0,
    sep=";")
sds.columns=sds.columns.map(int)

In [35]:
#DEFINE PARAMETERS

pm={#COVID PERIOD
    "covid":[2020,2021,2022],
    #INTENSITY UNIT CONVERSION
    #INTENSITY=IUC*EMISSIONS/ACTIVITY (IN SECTOR DATA UNITS)
    "iuc":1e3,
    #TTW TO WTW CONVERSION FACTOR
    "ttw2wtw":89.7/71.5,
    #YEAR AXIS
    "year":pd.Series(range(2015,2051)).set_axis(range(2015,2051))}

# TARGET MODEL (FUNCTIONS)

In [36]:
#FUNCTION - CHECK IF USER INPUTS SATISFY IMPOSED CONDITIONS

def user_inputs_check():
  if not basic_info_input.result \
  or not by_emissions_input.result \
  or not by_activity_input.result \
  or not ty_activity_input.result:
    user_inputs_valid=False
  else:
    user_inputs_valid=True
  return user_inputs_valid

In [37]:
#FUNCTION - CAPTURE USER INPUTS INTO DICTIONARY

def user_inputs_capture():
  return {"by":basic_info_input.children[0].value,
          "ty":basic_info_input.children[1].value,
          "ed_type":by_emissions_input.children[0].value,
          "cfe":by_emissions_input.children[1].value,
          "by_cfa":by_activity_input.children[0].value,
          "af_type":ty_activity_input.children[0].value,
          "cagr":ty_activity_input.children[1].value,
          "ty_cfa":ty_activity_input.children[2].value}

In [38]:
#FUNCTION - CONVERT USER INPUTS TO FORM REQUIRED BY MODEL

def user_inputs_convert(ip):
  #CONVERT EMISSIONS FROM TTW TO WTW (IF APPLICABLE)
  if ip["ed_type"]=="ttw":
    ip["cfe"]*=pm["ttw2wtw"]

  #CONVERT EMISSIONS FROM TCO2E TO MTCO2E (I.E. SECTOR DATA UNIT)
  ip["cfe"]*=1e-6

  #CONVERT PERCENTAGES TO PROPORTION
  ip["cagr"]*=.01

  #CONVERT ACTIVITY FROM RTK TO BLN RTK (I.E. SECTOR DATA UNIT)
  ip["by_cfa"]*=1e-9
  ip["ty_cfa"]*=1e-9

  return ip

In [39]:
#FUNCTION - SECTOR DECARBONIZATION APPROACH

def sda(
    #SECTOR OR SECTOR SEGMENT NAME (IN SECTOR DATA)
    name=None,
    #BASE YEAR
    by=None,
    #TARGET YEAR
    ty=None,
    #BASE YEAR EMISSIONS (IN SAME UNIT AS SECTOR DATA)
    bye=None,
    #ACTIVITY FORECAST (IN SAME UNIT AS SECTOR DATA)
    af=None,
    #SECTOR DATA
    sector_data=None,
    #INTENSITY UNIT CONVERSION
    #INTENSITY=IUC*EMISSIONS/ACTIVITY (IN SECTOR DATA UNITS)
    iuc=None):

  #SECTOR EMISSIONS, ACTIVITY & INTENSITY
  output=sector_data.loc[sector_data.index.str.contains(name)]
  for i in ["Emissions","Activity","Intensity"]:
    _=output.index[output.index.str.contains(name+" "+i)].values[0]
    output=output.rename(index={_:"Sector "+i})

  #COMPANY ACTIVITY
  output.loc["Company Activity"]=af

  #SDA (EQUATION 6 IN SDA REPORT)
  #SI TERM
  si=output.loc["Sector Intensity",2050]
  #P TERM
  p=output.loc["Sector Intensity"]-si
  p/=output.loc["Sector Intensity",by]-si
  #D TERM
  if output.loc["Company Activity",by]!=0:
    d=iuc*bye/output.loc["Company Activity",by]-si
  else:
    d=-si
  #BASE YEAR INTENSITY > CONVERGENCE INTENSITY
  if d>0:
    set_target=True
  #BASE YEAR INTENSITY <= CONVERGENCE INTENSITY
  else:
    set_target=False
  #M TERM
  m=output.loc["Company Activity",by]/output.loc["Sector Activity",by]
  m/=output.loc["Company Activity"]/output.loc["Sector Activity"]
  m[m>1]=1

  #COMPANY INTENSITY
  if set_target:
    output.loc["Company Intensity"]=d*p*m+si
  else:
    output.loc["Company Intensity"]=0
  output.loc["Company Intensity"]=output.loc["Company Intensity"].fillna(0)

  #COMPANY EMISSIONS
  output.loc["Company Emissions"]=\
  output.loc["Company Intensity"]*output.loc["Company Activity"]/iuc
  output.loc["Company Emissions"]=output.loc["Company Emissions"].fillna(0)

  #MASK YEAR<BASE YEAR
  output.loc[output.index.str.contains("Company"),output.columns<by]=np.nan

  #SORT
  output=output.sort_index()

  return output

In [40]:
#FUNCTION - TARGET MODEL

def target_model():

  #ACTIVITY FORECAST
  if ip["af_type"]=="cagr":
    af=ip["by_cfa"]*(1+ip["cagr"])**(pm["year"].astype(float)-ip["by"])
  elif ip["af_type"]=="tya":
    af=ip["by_cfa"]
    af+=(pm["year"]-ip["by"])*(ip["ty_cfa"]-ip["by_cfa"])/(ip["ty"]-ip["by"])

  #SDA TARGET BASED ON PACE
  output=sda(name="DFL",
             by=ip["by"],
             ty=ip["ty"],
             bye=ip["cfe"],
             af=af,
             sector_data=pace,
             iuc=pm["iuc"])

  #MASK COVID
  if ip["by"]<min(pm["covid"]):
    output[pm["covid"]]=np.nan

  return output

# RESULTS (FUNCTIONS)

In [41]:
#FUNCTIONS USED IN COMPANY_TARGET_PLOT

#LABEL BASE YEAR & TARGET YEAR
def label_by_ty(series=None,ax=None):
  x=ip["by"]
  y=series[ip["by"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Base Year",color="blue",verticalalignment="bottom")
  x=ip["ty"]
  y=series[ip["ty"]]
  ax.scatter(x,y,color="blue")
  ax.text(x,y," Target Year",color="blue",verticalalignment="bottom")

#COVID PERIOD (INTERPOLATE & DOTTED)
def covid_period(series=None,ax=None,color=None):
  if ip["by"]<min(pm["covid"]):
    _=series
    _=_.interpolate(method="linear",limit_area="inside")
    ax.plot(_[[min(pm["covid"])-1]+pm["covid"]+[max(pm["covid"])+1]],
            color=color,
            ls="dotted")

In [42]:
#FUNCTION - COMPANY TARGET PLOT

def company_target_plot():
  #FETCH DATA (EXCLUDE YEAR<BASE YEAR)
  chart=target.iloc[:,target.columns>=ip["by"]]

  #SETUP FIGURE
  fig,ax=plt.subplots()
  fig.set_size_inches(5,4)

  #COMPANY & SECTOR INTENSITY PLOT
  ax.plot(chart.loc["Sector Intensity"],
          color="orange",
          label="Sector Intensity")
  ax.plot(chart.loc["Company Intensity"],
          color="blue",
          label="Company Intensity")
  #LABEL BASE YEAR & TARGET YEAR
  label_by_ty(series=chart.loc["Company Intensity"],ax=ax)
  #COVID PERIOD (INTERPOLATE & DOT)
  covid_period(series=chart.loc["Sector Intensity"],
               ax=ax,
               color="orange")
  covid_period(series=chart.loc["Company Intensity"],
               ax=ax,
               color="blue")
  #FORMAT
  ax.grid()
  ax.legend()
  ax.set_title("Company Intensity Target")
  ax.set_xlabel("Year")
  ax.set_ylabel("Intensity (gCO2e/RTK)")

  plt.show(block=False)

In [43]:
#FUNCTION - TARGET TABLE

def target_table():
  #FETCH BASE YEAR & TARGET YEAR DATA
  table=target.loc["Company Intensity",[ip["by"],ip["ty"]]]
  table=table.to_frame().transpose()

  #% REDUCTION
  table["% Reduction ("+str(ip["by"])+"-"+str(ip["ty"])+")"]=\
  -1e2*(table[ip["ty"]]/table[ip["by"]]-1)

  #RENAME ROWS
  table.index=["Company Intensity (gCO2e/RTK)"]

  #RENAME COLUMNS
  table=table.rename({ip["by"]:"Base Year ("+str(ip["by"])+")",
                      ip["ty"]:"Target Year ("+str(ip["ty"])+")"},
                     axis=1)

  #ROUNDING
  table.iloc[:,:2]=table.iloc[:,:2].apply(lambda x:round(x,2))
  table.iloc[:,2]=table.iloc[:,2].apply(lambda x:round(x))

  #PERCENTAGE SIGN
  table.iloc[:,2]=table.iloc[:,2].astype(int).astype(str)+"%"

  #FORMAT DISPLAY
  table=table.style
  #NUMBER OF DECIMAL PLACES
  table=table.format(precision=2)
  #CENTRE ALIGN
  table=table.set_properties(**{"text-align":"center"})
  #COLUMN HEADING - MAX WIDTH 100PX & CENTRE ALIGN
  #ROW HEADING - RIGHT ALIGN
  table=table.set_table_styles(
      [dict(selector=".col_heading",
            props=[("max-width","100px"),("text-align","center")]),
       dict(selector=".row_heading",
            props=[("text-align","right")])])

  display(table)

# FINAL PRODUCT

In [44]:
#BUTTON WIDGET
calculate_target_button=widgets.Button(description="Calculate Target")

#OUTPUT WIDGET
results=widgets.Output()

#SPACER WIDGET
spacer=widgets.HTML(value="<br>")

#TARGET MODEL & RESULTS
@results.capture(clear_output=True)
def target_model_results(_):
  #REGISTER FIRST CLICK
  fc_widget.value=True
  #USER INPUTS DO NOT MEET IMPOSED CONDITIONS
  if not user_inputs_check():
    #OUTPUT ERROR MESSAGE
    display(user_inputs_error)
  #USER INPUTS MEET IMPOSED CONDITIONS
  else:
    #TARGET MODEL
    global ip,target
    ip=user_inputs_convert(user_inputs_capture())
    target=target_model()
    #CHECK IF MODEL RETURNED TARGET
    if target.loc["Company Intensity",ip["by"]]>0:
      no_target=False
    else:
      no_target=True
    #OUTPUT RESULTS
    if no_target:
      display(no_target_error)
    else:
      display(results_header)
      display(spacer)
      company_target_plot()
      display(spacer)
      target_table()

#LINK BUTTON TO FUNCTION
calculate_target_button.on_click(target_model_results)

#DISPLAY
_=VBox([
    #USER INPUT FORM
    overall_header,
    basic_info_header,
    basic_info_input,
    by_emissions_header,
    by_emissions_input,
    by_activity_header,
    by_activity_input,
    ty_activity_header,
    ty_activity_input,
    ty_activity_footer,
    #TARGET MODEL & RESULTS
    calculate_target_button,
    results],
    layout={"width":"1110px"})
display(_)